In [9]:
import math
import heapq

In [10]:
%run helpers.ipynb

In [11]:
def get_manhattan_distance(node_coord, destination_coord): #heuristic function
    ##TO EDIT
    dist = abs(destination_coord[0] - node_coord[0]) + abs(destination_coord[1] - node_coord[1])
    return dist

In [12]:
def reconstruct_graph(source, destination, parent):

    reconstructed_graph = []
    prev_node = destination #trace back from last node

    while prev_node != source:
        reconstructed_graph.append(prev_node)
        prev_node = parent[prev_node] #find previous using parent
    reconstructed_graph.append(source)

    reconstructed_graph = reconstructed_graph[::-1] #reverse order so that path is from src to destination
    
    return reconstructed_graph

In [16]:
def a_star_search(graph, coordinates, source, destination):
    g_score = {}
    parent = {} #keep track of parent node of each node
    
    for key, _ in graph.items():
        g_score[key] = math.inf
        parent[key] = ""
        
    g_score[source] = 0
    explored = set()
    heap = []
    # Push the source and its distance to the min heap or priority queue.
    heapq.heappush(heap, (0, source))
        
    while heap:
        _, node = heapq.heappop(heap)
        if node == destination:
            reconstructed_graph = reconstruct_graph(source, destination, parent)
            return reconstructed_graph, g_score[node]
        
        if node in explored:
            continue
        explored.add(node)
        
        for connected_node, cost in graph[node]:
            if connected_node in explored:
                continue
            
            g_cost = g_score[node] + cost # cost from the source node to current connected node
            h_cost = get_manhattan_distance(coordinates[connected_node], coordinates[destination])
            f_cost = g_cost + h_cost
            
            if g_cost < g_score[connected_node]:
                g_score[connected_node] = g_cost
                parent[connected_node] = node
                heapq.heappush(heap, (f_cost, connected_node)) 
                
    return None


In [19]:
def exhaustive_a_star_search(graph, coordinates, obstacles):
    min_total_g_score = math.inf
    best_path = []
    goal_permutations = permutate_goals(obstacles)
    
    for permutation in goal_permutations:
        print("-------------------------------")
        print("For permutation",  permutation)
        
        complete_path, total_g_score = a_star_search(graph, coordinates, 'A-0', permutation[0])
        print("g-score from node {source} to node {destination} is: {g_score}".format(
                source="A-0", destination=permutation[0], g_score=total_g_score))
        print("updated g-score: ", total_g_score)
        print("updated path: ", complete_path)
        
        for i in range(len(permutation)-1):
            partial_path, g_score = a_star_search(graph, coordinates, permutation[i], permutation[i+1])
            print("g-score from node {source} to node {destination} is: {g_score}".format(
                source=permutation[i], destination=permutation[i+1], g_score=g_score))
            
            total_g_score += g_score
            print("updated g-score: ", total_g_score)
            
            complete_path += partial_path
            print("updated path: ", complete_path)
        
        if total_g_score < min_total_g_score:
            min_total_g_score = total_g_score
            best_path = complete_path
    
    print("********************")
    print("best path:", best_path)
    print("min total g-score:", min_total_g_score)
    return best_path, min_total_g_score


print(exhaustive_a_star_search(GRAPH, COORDINATES, OBSTACLES))